**Import Libraries**

In [1]:
import tensorflow as tf
import numpy as np
import warnings
import os
import random
random.seed(42)
np.random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
tf.random.set_seed(42)
from nlp_model_text_preprocessing import index_the_words, text_to_sequence, pad_sequences, one_hot_encoding, index_the_char, text_to_sequence_char, char_sequence_to_text, word_sequence_to_text
from english_text_normalization import text_normalization
from deep_learning import nlp_model_word, nlp_model_char, model_compile_word, model_compile_char, model_fit, plot_word_model_change, plot_char_model_change
from model_testing import model_testing_char, model_testing_word
from transformers_models import load_dataset, data_collator, train_arguments, training_, save_model_tokenizer, transformer_testing,transformes_model
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

**Ignore Warnings**

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings(action='ignore')

**Read Data**

In [3]:
Corpus = open("The Alchemist.txt", encoding='utf-8-sig').read().lower().split("\n")
Corpus = [line.strip() for line in Corpus if line.strip()]
Corpus =  list(map(text_normalization, Corpus))
len(Corpus)

3668

In [4]:
with open('Cleaned_Corpus.txt', 'w') as f:
    for line in Corpus:
        f.write(line + '\n') 

**1-Tokens Based On Word**

In [ ]:
# Generate Word To Index, Index To Word, And Find Count Of All Words
all_words, words_index, index_to_words = index_the_words(Corpus)
all_words

In [ ]:
# Generate Input Sequences And Get The Max Length
input_sequence_ = []
for line in Corpus:
    token = text_to_sequence(words_index, line)
    for i in range(1, len(token)):
        n_grams = token[:i+1]
        input_sequence_.append(n_grams)
        
max_length_word =  max([len(x) for x in input_sequence_])
max_length_word

In [ ]:
# Padding The The Input Sequence To Make All Sequence In Same Length
input_sequence_ = pad_sequences(input_sequence=input_sequence_, max_length=max_length_word)
input_sequence_

In [ ]:
# Generate Train And Label Data For Model
train, labels = input_sequence_[:,:-1], input_sequence_[:,-1]

In [ ]:
# Convert Label Data 
label = one_hot_encoding(labels, all_words)
label

*Deep Learning Models*

In [ ]:
# Generate Early Stop Depending On Value Of Loss
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=5,
    mode = 'min',
    restore_best_weights=True,
)

In [ ]:
# 1-LSTM
lstm_model = tf.keras.layers.LSTM(units=128, return_sequences=False)
LSTM_1 = nlp_model_word(input_dim = all_words, output_dim = 100, input_length = max_length_word, unit = all_words, model = lstm_model)
model_compile_word(model =LSTM_1, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
history = model_fit(model=LSTM_1, Data=train, Label=label, epochs=150, early_stop=early_stop, batch_size=32)
plot_word_model_change(history=history)

In [ ]:
# 2-Bidirectional LSTM
bidirectional_lstm_model = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units= 128, return_sequences=False))
Bidirectional_LSTM_1 = nlp_model_word(input_dim = all_words, output_dim = 100, input_length = max_length_word, unit = all_words, model = bidirectional_lstm_model)
model_compile_word(model =Bidirectional_LSTM_1, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
history = model_fit(model=Bidirectional_LSTM_1, Data=train, Label=label, epochs=150, early_stop=early_stop, batch_size=64)
plot_word_model_change(history=history)

In [ ]:
# 3-GRU
gru_model = tf.keras.layers.GRU(units= 128, return_sequences=False)
GRU_1 = nlp_model_word(input_dim = all_words, output_dim = 100, input_length = max_length_word, unit = all_words, model = gru_model)
model_compile_word(model =GRU_1, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
history = model_fit(model=GRU_1, Data=train, Label=label, epochs=150, early_stop=early_stop, batch_size=32)
plot_word_model_change(history=history)

*Testing The Model*

In [ ]:
# 1-LSTM
model_testing_word(text = ' He decided to wait until the sun had', number_of_words = 25, text_normalization = text_normalization, text_to_sequence = text_to_sequence, words_index = words_index, pad_sequences = pad_sequences, model = LSTM_1, max_length = max_length_word, index_to_words = index_to_words, word_sequence_to_text = word_sequence_to_text, all_words=all_words)

In [ ]:
# 2-Bidirectional LSTM
model_testing_word(text = ' He decided to wait until the sun had', number_of_words = 25, text_normalization = text_normalization, text_to_sequence = text_to_sequence, words_index = words_index, pad_sequences = pad_sequences, model = Bidirectional_LSTM_1, max_length = max_length_word, index_to_words = index_to_words, word_sequence_to_text = word_sequence_to_text, all_words=all_words)

In [ ]:
# 3-GRU
model_testing_word(text = ' He decided to wait until the sun had', number_of_words = 25, text_normalization = text_normalization, text_to_sequence = text_to_sequence, words_index = words_index, pad_sequences = pad_sequences, model = GRU_1, max_length = max_length_word, index_to_words = index_to_words, word_sequence_to_text = word_sequence_to_text, all_words=all_words)

*Save The Models*

In [ ]:
LSTM_1.save('English_LSTM_1.h5')

In [ ]:
Bidirectional_LSTM_1.save('English_Bidirectional_LSTM_1.h5')

In [ ]:
GRU_1.save('English_GRU_1.h5')

**2-Token Based On Character**

In [5]:
# Generate Char To Index, Index To Char, And Find Count Of All Char
all_chars, chars_index, index_chars = index_the_char(Corpus)

In [6]:
# Find The Max Length
max_length_char = max([len(s) for s in Corpus])
max_length_char

123

In [7]:
# Generate Train And Label Data
Text = " ".join(Corpus)
Train = []
Label = []
for i in range(0, len(Text) - max_length_char):
    Train.append(Text[i: i + max_length_char])
    Label.append(Text[i + max_length_char])

In [ ]:
# Convert Words Into Number
sequence_text_char_Train = text_to_sequence_char(chars_index, Train)
sequence_text_char_Label = text_to_sequence_char(chars_index, Label)

In [ ]:
# Padding The The Input Sequence To Make All Sequence In Same Length
Train = pad_sequences(input_sequence=sequence_text_char_Train, max_length=max_length_char, padding='post')
# # Convert The Label Data
Label = one_hot_encoding(sequence_text_char_Label, all_chars)

**Deep Learning Model**

In [ ]:
# 1-LSTM
lstm_model = tf.keras.layers.LSTM(units= 128, return_sequences=False)
LSTM_2 = nlp_model_char(input_dim = all_chars, output_dim = 100, unit = all_chars, model = lstm_model)
model_compile_char(model =LSTM_2, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy())
history = model_fit(model=LSTM_2, Data=Train, Label=Label, epochs=150, early_stop=early_stop, batch_size=32)
plot_char_model_change(history=history)

In [ ]:
# 2-Bidirectional_LSTM
bidirectional_lstm_model = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units= 128, return_sequences=False))
Bidirectional_LSTM_2 = nlp_model_char(input_dim = all_chars, output_dim = 100, unit = all_chars, model = bidirectional_lstm_model)
model_compile_char(model =Bidirectional_LSTM_2, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy())
history = model_fit(model=Bidirectional_LSTM_2, Data=Train, Label=Label, epochs=150, early_stop=early_stop, batch_size=32)
plot_char_model_change(history=history)

In [ ]:
# 3-GRU
gru_model = tf.keras.layers.GRU(units= 128, return_sequences=False)
GRU_2 = nlp_model_char(input_dim = all_chars, output_dim = 100, unit = all_chars, model = gru_model)
model_compile_char(model =GRU_2, optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy())
history = model_fit(model=GRU_2, Data=Train, Label=Label, epochs=150, early_stop=early_stop, batch_size=32)
plot_char_model_change(history=history)

**Model Testing**

In [ ]:
# 1-LSTM
model_testing_char(text = 'He decided', number_of_chars = 100, char_index = chars_index, pad_sequences = pad_sequences, model = LSTM_2, max_length = max_length_char, index_char = index_chars, text_to_sequence_char = text_to_sequence_char, all_chars=all_chars, char_sequence_to_text=char_sequence_to_text)

In [ ]:
# 2-Bidirectional LSTM
model_testing_char(text = 'He decided', number_of_chars = 100, char_index = chars_index, pad_sequences = pad_sequences, model = Bidirectional_LSTM_2, max_length = max_length_char, index_char = index_chars, text_to_sequence_char = text_to_sequence_char, all_chars=all_chars, char_sequence_to_text=char_sequence_to_text)

In [ ]:
# 3-GRU
model_testing_char(text = 'He decided', number_of_chars = 100, char_index = chars_index, pad_sequences = pad_sequences, model = GRU_2, max_length = max_length_char, index_char = index_chars, text_to_sequence_char = text_to_sequence_char, all_chars=all_chars, char_sequence_to_text=char_sequence_to_text)

**Save The Model**

In [ ]:
LSTM_2.save('English_LSTM_2.h5')

In [ ]:
Bidirectional_LSTM_2.save('English_Bidirectional_LSTM_2.h5')

In [ ]:
GRU_2.save('English_GRU_2.h5')

**3-Transformers**

In [ ]:
# Transformers Model Name
import torch
device = torch.device("cpu")
model_name_ = 'gpt2'
# Model And Tokenizer For Transformers
tokenizer, model = transformes_model(model_name_)
model.to(device)
# Data Used For Train Transformers
train_dataset = load_dataset('Cleaned_Corpus.txt', tokenizer)
# Convert Data Into Batches
collator = data_collator(tokenizer)
# Set Arguments For Train The Transformers
train_args = train_arguments(epochs = 1)
# Train The Transformers
training_(model=model.to(device), training_args=train_args, collator = collator, data = train_dataset)
# Save The Model
save_model_tokenizer(model = model, tokenizer = tokenizer)
# Testing The Transformers Model
transformer_testing(input_text='He decided to wait until the sun ', tokenizer=tokenizer, model=model, text_normalization = text_normalization)
 